<h1 align="center">Traffic Sign Recognition</h1>

## Importing Libraries

In [1]:
# Libraries 
from flask import Flask,render_template,request
from pathlib import Path 
import matplotlib.pyplot as plt
import numpy as np 
from PIL import Image
import os
import pandas as pd 
import tensorflow as tf
from keras.utils import to_categorical
import cv2

### Reading the Input Image

In [64]:
X = pd.read_csv("Train.csv")
labels = X['Path'].values
y_train = X['ClassId'].values
print(labels.shape)
print(y_train.shape)

data=[]

for f in labels:
    image=cv2.imread('input/Train/'+f.replace('Train/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((30, 30))
    data.append(np.array(size_image))

X_train = np.array(data)
X_train = X_train.astype('float32')/255 

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 10)

(39209,)
(39209,)


In [65]:
print(X_train.shape)
print(y_train.shape)

(31367, 30, 30, 3)
(31367,)


### Defining Dense Neural Network

In [66]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compilation of the model
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='rmsprop', 
    metrics=['accuracy']
)

In [67]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 26, 26, 32)        2432      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 5, 5, 64)        

### Training the Model

In [68]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_train, y_train))
model.save('my_model.h5')

Epoch 1/10
981/981 [==============================] - 59s 60ms/step - loss: 1.1117 - accuracy: 0.6907 - val_loss: 0.1750 - val_accuracy: 0.9596
Epoch 2/10
981/981 [==============================] - 58s 59ms/step - loss: 0.1905 - accuracy: 0.9446 - val_loss: 0.0607 - val_accuracy: 0.9856
Epoch 3/10
981/981 [==============================] - 59s 60ms/step - loss: 0.1227 - accuracy: 0.9662 - val_loss: 0.0231 - val_accuracy: 0.9938
Epoch 4/10
981/981 [==============================] - 63s 65ms/step - loss: 0.0909 - accuracy: 0.9754 - val_loss: 0.0236 - val_accuracy: 0.9938
Epoch 5/10
981/981 [==============================] - 65s 66ms/step - loss: 0.0813 - accuracy: 0.9777 - val_loss: 0.0256 - val_accuracy: 0.9931
Epoch 6/10
981/981 [==============================] - 64s 66ms/step - loss: 0.0781 - accuracy: 0.9802 - val_loss: 0.0108 - val_accuracy: 0.9970
Epoch 7/10
981/981 [==============================] - 64s 66ms/step - loss: 0.0731 - accuracy: 0.9811 - val_loss: 0.0073 - val_accuracy:

### Testing the Model

In [69]:
#Predicting with the test data
y_test=pd.read_csv("Test.csv")
labels=y_test['Path'].values
y_test=y_test['ClassId'].values

data=[]

for f in labels:
    image=cv2.imread('input/test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((30, 30))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 
pred = model.predict_classes(X_test)

### Accuracy 

In [70]:
#Accuracy with the test data
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.931353919239905